In [62]:
#%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

In [63]:
#from ipywidgets import interact, interactive
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
import ipywidgets as widgets
import asyncio
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
import plotly
import math
import time
%matplotlib ipympl

In [64]:
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 180  
plt.ioff()

In [65]:
def get_intersections(x0, y0, r0, x1, y1, r1):
    # circle 1: (x0, y0), radius r0
    # circle 2: (x1, y1), radius r1

    d=math.sqrt((x1-x0)**2 + (y1-y0)**2)
    
    # non intersecting
    if d > r0 + r1 :
        return None
    # One circle within other
    if d < abs(r0-r1):
        return None
    # coincident circles
    if d == 0 and r0 == r1:
        return None
    else:
        a=(r0**2-r1**2+d**2)/(2*d)
        h=math.sqrt(r0**2-a**2)
        x2=x0+a*(x1-x0)/d   
        y2=y0+a*(y1-y0)/d   
        x3=x2+h*(y1-y0)/d     
        y3=y2-h*(x1-x0)/d 

        x4=x2-h*(y1-y0)/d
        y4=y2+h*(x1-x0)/d
        
        return (x3, y3, x4, y4)


In [66]:
X_limits = [-16,0]
Y_limits = [-0.25, 0.25]
R_def = 370
R_conv = R_def*1.33
Fix_point = [-R_def, R_def]
t_jumps = [0,0,0,0,0,0,0,0,0,0,0,0, -0.003,-0.003,-0.003,-0.003, -0.003, -0.02, -0.02, -0.02, -0.02]
R_VTX = [2.5,5,11,15]
NN = 50

In [67]:
intersect0_0 = get_intersections(0,0,R_VTX[0],-R_VTX[0], R_conv,R_conv)
intersect1_0 = get_intersections(0,0,R_VTX[1],-R_VTX[0], R_conv,R_conv)
intersect1_1 = get_intersections(0,0,R_VTX[1],-R_VTX[0], -R_conv,R_conv)
intersect2_0 = get_intersections(0,0,R_VTX[2],-R_VTX[0], R_conv,R_conv)
intersect2_1 = get_intersections(0,0,R_VTX[2],-R_VTX[0], -R_conv,R_conv)
intersect3_0 = get_intersections(0,0,R_VTX[3],-R_VTX[0], R_conv,R_conv)
intersect3_1 = get_intersections(0,0,R_VTX[3],-R_VTX[0], -R_conv,R_conv)


delta = 10
theta1_1 = math.atan2(intersect1_1[3],intersect1_1[2])

In [68]:
theta = np.linspace( -5.5*np.pi/4 , -2.5*np.pi/4 , 150 )
  
a = np.cos( theta )
b = np.sin( theta )


def my_track(x, x0, y0, R):
    """
    Return a sine for x with angular frequeny w and amplitude amp.
    """
    return (y0 - ( R**2 - ( x - x0 )**2 )**0.5  )

def my_conv(x, x0, y0, R, a=0):
    """
    Return a sine for x with angular frequeny w and amplitude amp.
    """
    return (y0 - (1-a)* ( R**2 - ( x - x0 )**2 )**0.5  ) 

print(my_track(-17,0,0,29))

-23.49468024894146


In [80]:
anim_fig1, anim_ax1 = plt.subplots()
text_kwargs = dict(ha='center', va='center', fontsize=14, color='C0')

xsin1 = np.linspace(X_limits[0], 0, 100)
xsin2 = np.linspace(-R_VTX[0], 0, 100)
xsin3 = np.linspace(X_limits[0],-R_VTX[0], 100)
def animate(intime):
    t = int(intime/1)
    plt.cla()
    NN=50
    if t>8: NN=200
    anim_ax1.plot( R_VTX[0]*a, R_VTX[0]*b , linewidth=2 )
    anim_ax1.plot( R_VTX[1]*a, R_VTX[1]*b , linewidth=2 )
    anim_ax1.plot( R_VTX[2]*a, R_VTX[2]*b , linewidth=2 )
    anim_ax1.plot( R_VTX[3]*a, R_VTX[3]*b , linewidth=2 )

    if t>=0 and t<3:
        line2 = plt.plot(xsin2,-xsin2*0, 'g--', linewidth=3)
        line3 = plt.plot(xsin3,my_conv(xsin3,-R_VTX[0], R_conv,R_conv,0), linewidth=2)
        line4 = plt.plot(xsin3,my_conv(xsin3,-R_VTX[0],-R_conv,R_conv,2), linewidth=2)
    if t>0:
        if t<3:
            anim_ax1.plot(intersect1_0[2],intersect1_0[3], 'bo') 
            anim_ax1.plot(intersect3_1[0],intersect3_1[1], 'go') 
            anim_ax1.plot(intersect2_1[0],intersect2_1[1], 'go') 
            anim_ax1.plot(intersect2_0[2],intersect2_0[3], 'bo') 

        anim_ax1.plot(intersect0_0[2],intersect0_0[3], 'bo') 
        anim_ax1.plot(intersect1_1[0],intersect1_1[1], 'go') 
        anim_ax1.plot(intersect3_0[2],intersect3_0[3], 'bo') 
    if t>1:
        delta = 1
        theta1_1 = -3.
        line1 = plt.plot(xsin1,my_track(xsin1,-4.5,R_def-0.005,R_def), linewidth=2)

    #if t > 8:
    #    plt.text(-1., 0.059, '$DCA_T$ = 290 $\mu$m', **text_kwargs)



    anim_ax1.set_xlim(X_limits)
    anim_ax1.set_ylim(Y_limits)

    plt.xlabel("X, cm")
    plt.ylabel("Y, cm")
    plt.text(-10., 1.069, '$p_T$ = 1 GeV', **text_kwargs)
    anim_ax1.legend(['cor. DC track','def. DC track'])
    #anim_ax1.legend(line2, "cor DC track")
    #plt.xlim(- 2 * np.pi, 2 * np.pi)


matplotlib.animation.FuncAnimation(anim_fig1, animate, frames=5, interval=2000, repeat_delay = 500)

In [70]:
if False:
    ani = matplotlib.animation.FuncAnimation(anim_fig1, animate, frames=12, interval=500)
    ani.save(filename="output/association_conv.html", writer="html")

In [81]:
if True:
    ani = matplotlib.animation.FuncAnimation(anim_fig1, animate, frames=5, interval=2000, repeat_delay = 500)
    ani.save(filename="output/association_conv.gif", writer="pillow")

In [72]:
#!jupyter nbconvert --to html association_anime_zoom.ipynb --HTMLExporter.theme=dark --no-input

import ipywidgets as widgets
import asyncio

slider = widgets.IntSlider(
    value=5,
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

async def test(slider, output):
    i = 0
    while slider.value != 10:
        i = i+1
        await asyncio.sleep(1)
        output.update('test ' + str(slider.value) + ' - ' + str(i))
        x = slider.value 
        c1= single_DCA_draw(slider.value)[0]
        if i == 1: c1.Draw()
        else : c1.Update()

output = display("tbd", display_id=True)
display(slider)

#asyncio.create_task(test(slider, output))

In [73]:

from IPython.core.display import display, HTML
display(HTML('association_anime_zoom.html'))

/tmp/ipykernel_31108/1323057843.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
